In [1]:
%set_env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

model_id = "neuralmagic/Mistral-Nemo-Instruct-2407-FP8"
llm = LLM(model_id, max_model_len=16384)
tokenizer = AutoTokenizer.from_pretrained(model_id)

INFO 07-26 10:33:11 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='neuralmagic/Mistral-Nemo-Instruct-2407-FP8', speculative_config=None, tokenizer='neuralmagic/Mistral-Nemo-Instruct-2407-FP8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=fp8, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=neuralmagic/Mistral-Nemo-Instruct-2407-FP8, use_v2_block_manager=False, enable_prefix_caching=False)
INFO 07-26 10:33:12 model_runner.py:680] Starting to load model neuralmagic/Mi

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 07-26 10:33:15 model_runner.py:692] Loading model weights took 12.9013 GB
INFO 07-26 10:33:17 gpu_executor.py:102] # GPU blocks: 10016, # CPU blocks: 1638
INFO 07-26 10:33:18 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-26 10:33:18 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-26 10:33:25 model_runner.py:1181] Graph capturing finished in 7 secs.


In [3]:
"""
I will provide you a piece of text. Try to think of a question to which the piece of text would be a suitable answer.
```
{answer}
```

Explain your reasoning.
Then return your final question on a new line starting with "QUESTION: "
"""

'\nI will provide you a piece of text. Try to think of a question to which the piece of text would be a suitable answer.\n```\n{answer}\n```\n\nExplain your reasoning.\nThen return your final question on a new line starting with "QUESTION: "\n'

In [4]:
from datasets import load_dataset
from pathlib import Path

datadir = Path("data")
ds_answers = load_dataset(
    "json", data_files=str(datadir.joinpath("interim", "answers.jsonl"))
)["train"]

In [8]:
prompt_template = """
I will provide you a piece of text. Try to think of a question to which the piece of text would be a suitable answer.
```
{answer}
```

Explain your reasoning.
Then return your final question on a new line starting with "QUESTION: "
"""

def generate_prompt(paragraph):
    return tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "You are an imaginative assistant."},
            {
                "role": "user",
                "content": prompt_template.format(answer=paragraph),
            },
        ],
        tokenize=False,
        add_generation_prompt=True,
    )

test_prompt = generate_prompt(ds_answers[0]["paragraph"])

In [46]:
sampling_params = SamplingParams(max_tokens=8192)
prompts = [generate_prompt(rec["paragraph"]) for rec in ds_answers]
request_outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|███████| 10797/10797 [14:55<00:00, 12.05it/s, est. speed input: 3853.95 toks/s, output: 1603.68 toks/s]


In [47]:
llm_responses = [ro.outputs[0].text for ro in request_outputs]

In [48]:
import pandas as pd

pd.Series(llm_responses).apply(lambda x: x.split("\n")[-1]).apply(lambda x: x.split(" ")[0]).value_counts()

QUESTION:        7135
**QUESTION:**    1594
"QUESTION:        807
**QUESTION:       377
**Question:**      79
                 ... 
"If                 1
"As                 1
"WHERE              1
Or,                 1
(QUESTION:          1
Name: count, Length: 155, dtype: int64

In [49]:
df_llm = ds_answers.to_pandas()

In [50]:
df_llm["response"] = llm_responses

In [51]:
df_llm.head()

,prefix,paragraph_index,paragraph,response
0,N14342,178,This harangue did but increase their consterna...,**Reasoning:** This text describes a military ...
1,N20751,738,I was greatly edified the other day in paying ...,"Based on the text, an appropriate question cou..."
2,N04486,77,"65. But in a Circle* drawn from one Body, as a...","Based on the provided text, it discusses the p..."
3,N19918,280,"What you offer, replied I, is nothing but soph...",The author's reasoning seems to be a response ...
4,N04182,136,"Having thus, in as plain a Manner as I could, ...",**Reasoning:**\n\nThe text discusses the autho...


In [52]:
df_llm["response"].apply(lambda x: x.split("\n")[-1]).apply(lambda x: x.split(" ")[0])

0        **QUESTION:**
1            QUESTION:
2            QUESTION:
3            QUESTION:
4        **QUESTION:**
             ...      
10792        QUESTION:
10793       "QUESTION:
10794    **QUESTION:**
10795        QUESTION:
10796        QUESTION:
Name: response, Length: 10797, dtype: object

In [53]:
selected_rows = df_llm["response"].apply(lambda x: x.split("\n")[-1]).apply(lambda x: x.split(" ")[0]).isin(["QUESTION:", "**QUESTION:**"])

In [54]:
df_selected = df_llm[selected_rows].copy()

In [55]:
df_selected.head()

,prefix,paragraph_index,paragraph,response
0,N14342,178,This harangue did but increase their consterna...,**Reasoning:** This text describes a military ...
1,N20751,738,I was greatly edified the other day in paying ...,"Based on the text, an appropriate question cou..."
2,N04486,77,"65. But in a Circle* drawn from one Body, as a...","Based on the provided text, it discusses the p..."
3,N19918,280,"What you offer, replied I, is nothing but soph...",The author's reasoning seems to be a response ...
4,N04182,136,"Having thus, in as plain a Manner as I could, ...",**Reasoning:**\n\nThe text discusses the autho...


In [61]:
import re
question_pattern = re.compile(r'(?s).*?(?:\*\*QUESTION:\*\*|QUESTION:)\s*(.*)')

df_selected["question"] = df_selected["response"].apply(lambda x: question_pattern.search(x).group(1))

In [62]:
df_selected

,prefix,paragraph_index,paragraph,response,question
0,N14342,178,This harangue did but increase their consterna...,**Reasoning:** This text describes a military ...,What did the American soldiers see happening w...
1,N20751,738,I was greatly edified the other day in paying ...,"Based on the text, an appropriate question cou...",What critique did the Quaker have about the Ep...
2,N04486,77,"65. But in a Circle* drawn from one Body, as a...","Based on the provided text, it discusses the p...",What is the gravitational force experienced by...
3,N19918,280,"What you offer, replied I, is nothing but soph...",The author's reasoning seems to be a response ...,What are the principles discussed in the text ...
4,N04182,136,"Having thus, in as plain a Manner as I could, ...",**Reasoning:**\n\nThe text discusses the autho...,"What misconceptions about the nature of a ""Wor..."
...,...,...,...,...,...
10791,N15939,15,"THE text therefore, and the great occasion, on...",Reasoning: The provided text appears to be an ...,How does the speaker believe a strong civil un...
10792,N06545,453,This Meeting (as before mentioned) being gener...,**Reasoning:**\n\nThe text appears to be a tra...,How did George Keith respond to the speaker's ...
10794,N27874,12,"The WHITE LOAF.\nONE day, it was about the mid...",**Explanation:**\n\nThe text describes a situa...,What had Mrs. White intended to use the small ...
10795,N17823,795,"Being now about church matters, I will here in...",The text describes a disagreement among church...,How was the dispute over the location of the n...


In [64]:
system_prompt = "You are a helpful assistant that answers in the style of early American written sources"

def row_to_jsonline(row):
    conversations = []
    conversations.append({"from": "system", "value": system_prompt})
    conversations.append({"from": "human", "value": row["question"]})
    conversations.append({"from": "gpt", "value": row["paragraph"]})
    return {"conversations": conversations}

df_selected.apply(row_to_jsonline, axis=1)
    

0        {'conversations': [{'from': 'system', 'value':...
1        {'conversations': [{'from': 'system', 'value':...
2        {'conversations': [{'from': 'system', 'value':...
3        {'conversations': [{'from': 'system', 'value':...
4        {'conversations': [{'from': 'system', 'value':...
                               ...                        
10791    {'conversations': [{'from': 'system', 'value':...
10792    {'conversations': [{'from': 'system', 'value':...
10794    {'conversations': [{'from': 'system', 'value':...
10795    {'conversations': [{'from': 'system', 'value':...
10796    {'conversations': [{'from': 'system', 'value':...
Length: 8729, dtype: object

In [66]:
import json
with datadir.joinpath("interim", "chatbot_train_data.jsonl").open("w", encoding="utf-8") as f:
    for example in df_selected.apply(row_to_jsonline, axis=1):
        f.write(json.dumps(example) + "\n")
        
    

In [68]:
!head -n1 data/interim/chatbot_train_data.jsonl | jq

{
  "conversations": [
    {
      "from": "system",
      "value": "You are a helpful assistant that answers in the style of early American written sources"
    },
    {
      "from": "human",
      "value": "What did the American soldiers see happening when they started firing their batteries?"
    },
    {
      "from": "gpt",
      "value": "This harangue did but increase their consternation. As soon as they began to\nhear the terrible roaring of our batteries, we that were on the heights saw\nthem flying precipitately from their redoubts, while their batteries in an\ninstant were entirely silenced. They had been quiet spectators of our labors,\nand we now became so in our turn with respect to them. At this time I watched\nan opportunity to traverse our lines, which consisted of a large ditch, broad\nenough for carriages to travel in, about four feet in depth, and covered by a\nrampart of gabions, or cylindrical baskets, fixed upon the ground, by means of\nprojecting stakes, filled

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
